In [29]:
import pandas as pd
import numpy as np
import sqlite3
import re

conn = sqlite3.connect("profiles.db")
cursor = conn.cursor()
cursor.execute("ATTACH DATABASE 'colleges.db' AS colleges")

In [30]:
df = pd.read_sql_query("SELECT * FROM colleges.university_profiles AS c JOIN profiles AS p ON c.school = p.school",conn)
df_w_admission_rate = df.copy(deep=True)

In [31]:
def get_type_map(df_column):
    return {t:i for i, t in enumerate(set(df_column))}

def convert_sat_to_act(sat):
    if np.isnan(sat):
        return np.nan
    sat_scores = np.array([990, 1060, 1140, 1210, 1270, 1330, 1390, 1450, 1510, 1560, 1620, 1680, 1740, 1800, 1860, 1920, 1980, 2020, 2080, 2140, 2220, 2290, 2380, 2410])
    return np.where(sat < sat_scores)[0][0] + 13
    
def combine_test_scores(x):
    if not np.isnan(x['act']):
        return x['act']/36.0
    sat = np.array([x['sat_m'], x['sat_r'], x['sat_w']])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def convert_string(x):
    try:
        return int(x)
    except:
        return np.nan

def combine_avg_test_scores(x):
    if not np.isnan(convert_string(x['avg_act'])):
        return convert_string(x['avg_act'])/36.0
    sat = np.array([convert_string(x['sat_math']), convert_string(x['sat_reading']), convert_string(x['sat_writing'])])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def map_statuses(x):
    status_map = {"Denied": 0, "Deferred": 0, "Wait-Listed": 0, "Accepted": 2, "Will Attend": 2}
    return status_map[x['status']]

def convert_class_rank(x):
    s = str(x['class_rank'])
    if " of " in s:
        return float(s[:s.index(" of ")])/float(s[s.index(" of ") + 4:])
    if "Top " in s:
        return float(s[4:s.index("%")])/100.0
    if "Bottom " in s:
        return float(s[7:s.index("%")])/100.0
    return np.nan

def convert_instate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        instate = x['cost_attendance'].split("<br>")[0]
        return instate[instate.index("$")+1:]
    else:
        return x['cost_attendance']

def convert_outstate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        outstate = x['cost_attendance'].split("<br>")[1]
        return outstate[outstate.index("$")+1:]
    else:
        return x['cost_attendance']

def get_tuition(x):
    return x['in_state_tuition'] if x['state'] == x['hs_state'] else x['out_state_tuition']

def map_hs_types(x):
    hs_type_map = {"Public": 0, "Private": 1, "Parochial": 2, "Home": 3}
    return hs_type_map[x['hs_type']]

def map_ins_types(x):
    ins_type_map = {"Public": 0, "Private": 1, "Private for-profit": 1, "&nbsp;": np.nan}
    return ins_type_map[x['institution_type']]

In [32]:
df = df.loc[:,~df.columns.duplicated()]
df.fillna(value=pd.np.nan, inplace=True)
df['avg_act'] = df['avg_act'].map(lambda x: str(x).strip('-'))
df['test_score'] = df.apply(combine_test_scores, axis=1)
df['avg_test_score'] = df.apply(combine_avg_test_scores, axis=1)
df['class_rank'] = df.apply(convert_class_rank, axis=1)
df['institution_type'] = df.apply(map_ins_types, axis=1)
df['in_state_tuition'] = df.apply(convert_instate_tuition, axis=1)
df['out_state_tuition'] = df.apply(convert_outstate_tuition, axis=1)
#df['tuition'] = df.apply(get_tuition, axis=1)

for col in ['cost_attendance', 'city', 'sat_m', 'sat_r', 'sat_w', 'act', 'sat_math', 'sat_reading', 'sat_writing', 'avg_act', 'gpa_w']:
    df = df.drop(col, axis=1)

df = df.dropna()#subset=['test_score','avg_test_score','gpa_uw','class_rank','hs_state','hs_type','institution_type'])

df['status'] = df.apply(map_statuses, axis=1)
df['hs_type'] = df.apply(map_hs_types, axis=1)
replace_columns = ['hs_state', 'state', 'gender']
for col in replace_columns:
    df[col].replace(get_type_map(df[col]), inplace=True)

In [33]:
print(len(df.index))

44042


In [34]:
whole_df = df

def split_df_on(feature=None):
    split_dfs = {}
    if feature:
        for val in whole_df[feature].unique():
            split_dfs[val] = (whole_df.loc[whole_df[feature]==val])
    else:
        split_dfs = {'all': whole_df}
    return split_dfs
df.sample(n=100)



,school,state,avg_gpa,average_freshman_aid,admission_rate,faculty_total,international_percent,institution_type,female_percentage,year,...,gpa_uw,class_rank,status,eaed,legacy,athlete,test_score,avg_test_score,in_state_tuition,out_state_tuition
67812,"University of California, Los Angeles",26,4.36,24405,18,1570,11.,0.0,57.5,2018,...,4.00,0.003704,2,0,0,0,0.861111,0.805556,30485,58499
44452,Colorado State University,19,3.62,11572,83,1318,3.4,0.0,51.9,2012,...,2.90,0.500000,2,0,0,0,0.638889,0.694444,26462,43970
47865,Stanford University,26,3.94,53337,5,1613,9.1,1.0,48.5,2016,...,3.97,0.048485,0,1,0,0,0.944444,0.861111,66184,66184
30882,Hobart and William Smith Colleges,43,3.37,36231,61,221,6.2,1.0,50.8,2015,...,3.72,0.250000,2,0,0,0,0.777778,0.777778,71400,71400
18789,University of Massachusetts Amherst,29,3.89,17495,57,1351,5.9,0.0,49.8,2009,...,3.50,0.076000,2,0,0,0,0.861111,0.777778,30254,48320
39944,Montana State University,34,3.38,1154,83,600,3.3,0.0,45.0,2018,...,3.60,0.250000,2,0,0,0,0.750000,0.583333,21273,37977
46454,San Jose State University,26,3.40,19745,55,687,6.2,0.0,47.8,2013,...,3.60,0.051786,2,0,0,0,0.777778,0.555556,26278,35206
27182,University of Michigan,31,3.84,27050,29,2791,7.1,0.0,49.8,2013,...,4.00,0.018182,2,1,0,0,1.000000,0.805556,29526,62176
30722,University of Hartford,5,3.05,27225,81,355,5.1,1.0,51.1,2011,...,3.90,0.100000,2,0,0,0,0.805556,0.666667,56599,56599
58024,Georgia College and State University,33,3.49,11051,79,330,0.5,0.0,62.3,2012,...,3.30,0.067347,2,1,0,0,0.694444,0.694444,28120,46834


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression

split_on = None
for val, df_s in split_df_on(split_on).items():
    k = 1
    
    df0 = df_s[df.status == 0]
    df2 = df_s[df.status == 2].sample(n = int(k*len(df0)))
    
    df_final = pd.concat([df0, df2])
    schools = df_final['school']
    print(len(df_final[(df_final['school'] == 'Florida International University')]))
    df_final_copy = df_final.copy()
    df_final = df_final.drop('school', axis=1)
    X = df_final.drop('status', axis=1)
    y = df_final['status']
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    mlp = MLPClassifier(hidden_layer_sizes=(21, 21, 21),max_iter=1000)
    mlp.fit(X_train,y_train)
    predictions = mlp.predict(X_test)
    print(confusion_matrix(y_test,predictions))
    print(classification_report(y_test,predictions))

    logisticRegr = LogisticRegression()
    logisticRegr.fit(X_train, y_train)
    score = logisticRegr.score(X_test, y_test)
    print(f"Logistic Regression Score for {split_on} {val}: " + str(score))



48
[[1809  349]
 [ 520 1717]]
             precision    recall  f1-score   support

          0       0.78      0.84      0.81      2158
          2       0.83      0.77      0.80      2237

avg / total       0.80      0.80      0.80      4395

Logistic Regression Score for None all: 0.7802047781569966


In [36]:
old_x = X.copy()
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [37]:

from collections import defaultdict
# Experimental Acceptance Rate
e_a_r = defaultdict(lambda: defaultdict(int))
total_applicants = 0
for res, school in zip(mlp.predict(X), schools):
    if res == 0:
        e_a_r[school]['experimental_reject'] += 1
    elif res == 2:
        e_a_r[school]['experimental_accept'] += 1
    e_a_r[school]['total'] += 1
    total_applicants += 1

for school in schools.unique():
    for student in df_final_copy[df_final_copy['school'] == school].iterrows():
        if student[1][15] == 0:
            e_a_r[school]['actual_reject'] += 1
        else:
            e_a_r[school]['actual_accept'] += 1

print(total_applicants == len(schools))


True


In [38]:
e_a_r

defaultdict(<function __main__.<lambda>>,
            {'Abilene Christian University': defaultdict(int,
                         {'actual_accept': 2,
                          'experimental_accept': 2,
                          'total': 2}),
             'Adams State University': defaultdict(int,
                         {'actual_reject': 1,
                          'experimental_accept': 1,
                          'total': 1}),
             'Adelphi University': defaultdict(int,
                         {'actual_accept': 11,
                          'experimental_accept': 11,
                          'total': 11}),
             'Agnes Scott College': defaultdict(int,
                         {'actual_accept': 10,
                          'actual_reject': 1,
                          'experimental_accept': 11,
                          'total': 11}),
             'Alabama State University': defaultdict(int,
                         {'actual_accept': 4,
                          '

In [54]:
compare_vs_ground_truth = defaultdict(str)

for (index, row), school in zip(old_x.iterrows(), schools):
    accept_rate = None
    if e_a_r[school]['experimental_accept'] != 0 and e_a_r[school]['experimental_reject'] != 0:
        accept_rate = e_a_r[school]['experimental_accept']/e_a_r[school]['total']
    elif e_a_r[school]['experimental_accept'] == 0:
        accept_rate = 0
    elif e_a_r[school]['experimental_reject'] == 0:
        accept_rate = 1
    actual_accept_rate = None
    if e_a_r[school]['actual_accept'] != 0 and e_a_r[school]['actual_reject'] != 0:
        actual_accept_rate = e_a_r[school]['actual_accept']/e_a_r[school]['total']
    elif e_a_r[school]['actual_accept'] == 0:
        actual_accept_rate = 0
    elif e_a_r[school]['actual_rject'] == 0:
        actual_accept_rate = 1
    compare_vs_ground_truth[school] = [accept_rate, actual_accept_rate, int(row['admission_rate'])/100]

In [55]:
compare_vs_ground_truth

defaultdict(str,
            {'Abilene Christian University': [1, 1, 0.51],
             'Adams State University': [1, 0, 0.99],
             'Adelphi University': [1, 1, 0.73],
             'Agnes Scott College': [1, 0.9090909090909091, 0.66],
             'Alabama State University': [0.7142857142857143,
              0.5714285714285714,
              0.98],
             'Albany College of Pharmacy and Health Sciences': [1, 0.25, 0.67],
             'Albany State University': [0.4, 0.4, 0.5],
             'Albertus Magnus College': [1, 1, 0.67],
             'Albion College': [1, 1, 0.71],
             'Albright College': [1, 1, 0.5],
             'Alcorn State University': [1, 1, 0.79],
             'Alfred University': [0.875, 0.875, 0.63],
             'Allegheny College': [0.9444444444444444,
              0.8333333333333334,
              0.68],
             'Alma College': [1, 1, 0.64],
             'Alvernia University': [1, 0.8, 0.74],
             'American International Coll

In [56]:
avg_error = 0
error_rates = {}
for (school, admission_stats), (school_2, comparisons) in zip(e_a_r.items(), compare_vs_ground_truth.items()):
    if school != school_2:
        print("there's a huge mistake somewhere")
    error = abs(comparisons[1]-comparisons[0])
    weight_error = admission_stats['total']/total_applicants
    avg_error += error*weight_error
    if admission_stats['total'] > 25:
        error_rates[school] = {'error': error, 'experimental': comparisons[0], 'real': comparisons[1], 'university reported': comparisons[2]}
        error_rates[school].update(admission_stats)

In [57]:
avg_error

0.10056882821387907

In [58]:
sorted_errors = sorted(error_rates.items(), reverse=False, key=lambda item: item[1]['error'])
flatten_dict = []
for school, stats in sorted_errors:
    stats['school'] = school
    flatten_dict.append(stats)
flatten_dict

[{'accept': 0,
  'actual_accept': 60,
  'actual_reject': 71,
  'error': 0.0,
  'experimental': 0.4580152671755725,
  'experimental_accept': 60,
  'experimental_reject': 71,
  'real': 0.4580152671755725,
  'school': 'Binghamton University',
  'total': 131,
  'university reported': 0.4},
 {'accept': 0,
  'actual_accept': 20,
  'actual_reject': 10,
  'error': 0.0,
  'experimental': 0.6666666666666666,
  'experimental_accept': 20,
  'experimental_reject': 10,
  'real': 0.6666666666666666,
  'school': 'West Chester University of Pennsylvania',
  'total': 30,
  'university reported': 0.69},
 {'accept': 0,
  'actual_accept': 11,
  'actual_reject': 21,
  'error': 0.0,
  'experimental': 0.34375,
  'experimental_accept': 11,
  'experimental_reject': 21,
  'real': 0.34375,
  'school': 'Scripps College',
  'total': 32,
  'university reported': 0.33},
 {'accept': 0,
  'actual_accept': 8,
  'actual_reject': 35,
  'error': 0.0,
  'experimental': 0.18604651162790697,
  'experimental_accept': 8,
  'exp

In [59]:
output_df = pd.DataFrame(flatten_dict, columns=['school', 'error', 'experimental', 'real', 'experimental_a', 'reject', 'total'])
output_df

,school,error,experimental,real,accept,reject,total
0,Binghamton University,0.000000,0.458015,0.458015,0,NaN,131
1,West Chester University of Pennsylvania,0.000000,0.666667,0.666667,0,NaN,30
2,Scripps College,0.000000,0.343750,0.343750,0,NaN,32
3,Kenyon College,0.000000,0.186047,0.186047,0,NaN,43
4,Emory University,0.000000,0.402878,0.402878,0,NaN,139
5,University of Tennessee,0.000000,0.875000,0.875000,0,NaN,40
6,Iowa State University,0.000000,1.000000,1.000000,0,NaN,26
7,University of Kansas,0.000000,1.000000,1.000000,0,NaN,33
8,University of Vermont,0.008772,0.736842,0.728070,0,NaN,114
9,Bucknell University,0.010989,0.153846,0.142857,0,NaN,91
